# Working with Tensors in Tensorflow

Before you can start to follow this course, you should run the following command with your virtual python environment activated: "pip install numpy matplotlib tensorflow tensorflow-datasets keras-cv"

This is the first of a series of notebooks for a course on Deep Learning with TensorFlow. This notebook is advised to be reviewed carefully if you have not worked extensively with numpy and multi-dimensional arrays or tensors before. Most of the API is similar to numpy with the difference that you ought not use in place operations as TensorFlow is a functional DL framework. Instead of tensor.reshape(shape) you should use tf.reshape(tensor, shape). This can often lead to seemingly convoluted code in which you have to read from the inner-most bracket to the outermost bracked to understand what is going on.

# What is a tensor? 

- **A Tensor is a generalization of the concepts scalar, vector and matrix**

Rank 0 tensor: **scalar** with shape () e.g. 5

Rank 1 tensor: **vector** with shape (n) e.g. $\begin{pmatrix}1 \\ 2 \\ 3 \end{pmatrix}$


Rank 2 tensor: **matrix** with shape (m,n) e.g. $\begin{bmatrix}1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 \end{bmatrix}$

Rank 3 tensor: e.g. **image** with shape (width, height, color channels)

Rank 4 tensor: e.g. RGB video with shape (time, width, height, color channels)

Rank 5 tensor: e.g. batch of RGB videos with shape (batch_size, time, width, height, color channels)

.
.
.

# Creating tensors from other objects in Python

In [14]:
import tensorflow as tf
import numpy as np

to_tensor = 5.0

to_tensor = [ [1,2,3], \
              [4,5,6] ]

#to_tensor = np.arange(1,7)
#to_tensor = np.ones((2,3))
#to_tensor = np.random.randn(5,2,3)


tensor = tf.constant(to_tensor, 
                    dtype=tf.float32
                    )
print(tensor)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)


# Creating tensors directly with Tensorflow

In [15]:
tensor = tf.ones(shape=(3,2)) # tf.zeros # tf.zeros_like # tf.ones_like #tf.eye

tensor = tf.random.normal((3,2))

# generate tensor filled with random integers
#tensor = tf.random.uniform((4,3,2), minval= 0, maxval=100, dtype=tf.int32)

print(tensor)

tf.Tensor(
[[ 1.3696206  -1.3010486 ]
 [-0.44483045  0.7362469 ]
 [-0.07830498 -0.86602336]], shape=(3, 2), dtype=float32)


# Reshaping and transposing tensors

In [16]:
tensor = tf.range(1,101, dtype=tf.float32)
print(tensor.shape, "\n")

# reshaping
reshaped_tensor = tf.reshape(tensor, (20,5))
print(reshaped_tensor.shape, "\n")

# adding a dimension of 1: e.g. from tensor shape (32,10) -> (32,10,1)
expanded_tensor = tf.expand_dims(tensor, axis=-1)
print(expanded_tensor.shape, "\n")

# removing dimensions of 1: e.g. (1,20,1,20,1) -> (20,20)
squeezed_tensor = tf.squeeze(expanded_tensor, axis=None)
print(squeezed_tensor.shape, "\n")

# transposing

transposed_tensor = tf.transpose(reshaped_tensor)
print(transposed_tensor.shape, "\n")

# transposing with permutation: e.g. (10,5,2) -> (5,10,2)
permuted_tensor = tf.transpose(reshaped_tensor, perm=[1,0])
print(permuted_tensor.shape)

(100,) 

(20, 5) 

(100, 1) 

(100,) 

(5, 20) 

(5, 20)


# Concatenating and stacking tensors

Concatenating is something you do when the axis along which you want to stack already exists. The tensors need not to match in terms of elements in that axis. Example usage: Combine outputs from different layers

Stacking is something you do when you have multiple tensors of the exact same shape and you want to stack them in a new axis. Example: Turn multiple images into a batch or time-sequence of images.

In [17]:
A = tf.ones((128,32), dtype=tf.float32)
B = tf.zeros((128,64), dtype=tf.float32)

print(tf.concat([A,B], axis= -1).shape)


A = tf.ones((12,12,3), dtype=tf.float32)
B = tf.zeros((12,12,3), dtype=tf.float32)

print(tf.stack([A,B], axis=0).shape)

(128, 96)
(2, 12, 12, 3)


# Reduce-operations in tensorflow

- instead of sum, we use reduce_sum and specify the axis over which we want to sum. We can pass a list of axes to sum/mean/product/std etc. over, or we can pass axis=None, to reduce over all axes to obtain a single scalar.
- same for other operations like mean, std, product etc.
- set argument keepdims=True if you want to do something like dividing a tensor by its standard deviation.

In [18]:
tensor = tf.ones((32,4,2))
print(tensor.shape,"\n")

feature_sum = tf.reduce_sum(tensor, axis=1)
print(feature_sum.shape, "\n")

batch_sum = tf.reduce_prod(tensor, axis=0)
print(batch_sum.shape, "\n")

total_sum = tf.reduce_mean(tensor, axis=None)
print(total_sum.shape)

total_std = tf.math.reduce_std(tensor,axis=None)
print(total_std.shape)

(32, 4, 2) 

(32, 2) 

(4, 2) 

()
()


# Matrix multiplication and other operations with tensors

In Deep Learning we make extensive use of matrix multiplications. If you have not learned about linear algebra before, I recommend you watch [3Blue1Brown's series](https://www.youtube.com/watch?v=fNk_zzaMoSs&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab).

In [19]:
A = tf.ones((32,2,5))
B = tf.ones((32, 5,10))

result = tf.matmul(A,B)
#result = A@B
print(result.shape)

(32, 2, 10)


In [20]:
# element-wise operations

A = tf.range(1,6, dtype=tf.float32)
B = tf.ones(A.shape)*0.5 # broadcast multiplication by 0.5 
print(B*A)

tf.Tensor([0.5 1.  1.5 2.  2.5], shape=(5,), dtype=float32)


# Constants and variables

- Tensors can be constants or variables

- Constants should be used for data (immutable), variables for model parameters (mutable with variable.assign(...))

- Variables can be flagged as trainable or non-trainable

In [21]:
var = tf.Variable([1,2,3], trainable = True, dtype=tf.float32, name="weight_1")
print(var)

var.assign([2,3,4])
print(var)

<tf.Variable 'weight_1:0' shape=(3,) dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>
<tf.Variable 'weight_1:0' shape=(3,) dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>


# Converting current variable values to a constant tensor

- Can be useful for keeping track of model parameters during training

In [22]:
stored_tensor = tf.convert_to_tensor(var)
print(stored_tensor)
var.assign([9,9,9])
print(stored_tensor) # stayed the same even though var changed

tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)
tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)


# Converting tensors to a different dtype

We often need to change the data type of tensors. In TensorFlow this is done with tf.cast

In [23]:
# before transforming, we have float32
print(stored_tensor.dtype)

tf.cast(stored_tensor, dtype=tf.int8)

<dtype: 'float32'>


<tf.Tensor: shape=(3,), dtype=int8, numpy=array([2, 3, 4], dtype=int8)>

# Concluding remarks

The TensorFlow is quite extensive and it can feel overwhelming at first, especially if you are not very familiar with NumPy. It is worth to review the documentation for specific functions and to use stackoverflow and other resources extensively. Beware however that you may find outdated information regarding TensorFlow 1.x versions which include things such as tf.PlaceHolder and session.run() calls.